# **Machine Learning - Unsupervised Learning - 2**


## Advanced Anomaly Detection


## Temporal Anomaly Detection
We will do detection for temporal data. Recall that the challenge is the temporal data is a combination of seasonal patterns and/or cycles, an actual overall trend, and noise. So in order to do detection we must first take these into consideration and make our data stationary. This will allow us to we have an expectation for what we should see at even given time point, and a deviation (residual) capturing the distance between our expectation and observation.

### Seasonal and Trend decomposition using Loess
Seasonal and Trend decomposition using Loess (STL) is a very versatile and robust method for decomposing time series developed by [(Cleveland et al., 1900)](http://cs.wellesley.edu/~cs315/Papers/stl%20statistical%20model.pdf). In class we discussed [weighted and moving averages](https://en.wikipedia.org/wiki/Moving_average), but Loess is another technique for estimating nonlinear relationships via decomposition. Specifically, STL decomposes time series data into seasonal, trend, and irregular components using loess and plots the components separately. We will see this in action by exploring the `elecequip` which shows the number of new orders for electrical equipment (computer, electronic and optical products) in the Euro area (16 countries). The data have been adjusted by working days and normalized so a value of 100 corresponds to 2005.

In [ ]:
import numpy as np
import pandas as pd
from statsmodels.tsa.seasonal import STL

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
#%matplotlib inline

In [ ]:
elec_df = pd.read_csv("./data/elecequip.csv")
elec_df.head()

**NOTE**: 

1. We convert to date time format
2. Apply Month-Year as an index. 
3. Finally set the appropriate frequency. The frequency can be infered from the data. 

In [ ]:
elec_df['Month-Year'] = pd.to_datetime(elec_df['Month-Year'], format="%m/%d/%Y")

In [ ]:
elec_df.set_index(['Month-Year'], inplace=True)


In [ ]:
elec_df.index.freq = pd.infer_freq(elec_df.index)
elec_df.head()

In [ ]:
elec_df['Value'].plot()
plt.title("Electrical equipment manufacturing")
plt.ylabel("New orders")

### Now we fit the STL model to this time series data

In [ ]:
from statsmodels.tsa.seasonal import STL
stl = STL(elec_df['Value'], trend=13, seasonal = 5)
res = stl.fit()

### You can plot the trend line along with actual data


In [ ]:
figure, ax = plt.subplots()
res.trend.plot(ax=ax)
elec_df['Value'].plot(ax=ax)
plt.title("Electrical equipment manufacturing")
plt.ylabel("New orders")

The plot shows the trend component in black and the original data in ornage. The trend shows the overall movement in the series, ignoring the seasonality and noise components.

We can also show an additive decomposition of these data using STL by plotting the `fit` object

### You can visualize the trend, seasonality and residuals identified by STL method

In [ ]:
fig = res.plot()

where have the data decomposed into data overall trend, seasonal variation, and then left over noise. **The left over noise, known as residuals, can be evaluated for outliers**. We should look at the density noise component and see if it (mostly) look gaussian or not,if so then we can, check it for outliers using General Extreme Studentized Deviate.

In [ ]:
noise = res.resid
sns.kdeplot(noise)
plt.title("Density of the Noice")

## General Extreme Studentized Deviate
In our lecture we discussed [GESD](http://www.itl.nist.gov/div898/handbook/eda/section3/eda35h3.htm), which is a way to identify outliers in our data. 

### GESD Helper functions

Below is the helper function. **Read through it to see how it is working.**

In [ ]:
import math
import scipy.stats as stats
def gesd(obs, alpha, value_zscore = np.nan, r = np.nan):
    n = len(obs)
    if (pd.isna(r)): # by default, set upper bound on number of outliers 'r' to 1/2 sample size
        r = math.floor(n/2)
    R = np.zeros(shape = r) # test statistics for 'r' outliers
    lamb = np.zeros(shape = r) # critical values for 'r' outliers
    outlier_ind = np.zeros(shape = r) # removed outlier observation values
    outlier_val = np.zeros(shape = r) # removed outlier observation values
    
    m = 0   # number of outliers
    obs_new = np.array(obs) # temporary observation values
    
    ### Find outliers ####
    for i in range(r):
        
        #### Compute test statistic ####
        if (str.lower(value_zscore) == "yes"):
            z = abs(obs_new) # If Z-score is alrealy computed
        elif (str.lower(value_zscore) == "no"):
            z = abs(obs_new - np.mean(obs_new))/np.std(obs_new) # Z-scores
        else:
            print("ERROR! Inappropriate value for value.score=[YES|NO]")
        
        max_ind = np.argmax(z) # in case of ties, return first one
        R[i] = z[max_ind] # max Z-score
        
        outlier_val[i] = obs_new[max_ind] # removed outlier observation values
        outlier_ind[i] = max_ind # index of removed outlier observation values
        
        obs_new = np.delete(obs_new, max_ind) # remove observation that maximizes |x_i - x_mean|
        
        #### Compute critical values ####
        p = 1 - alpha/(2*(n-i+1)) # probability
        t_pv = stats.t.ppf(p, (n-i-1)) # Critical value from Student's t distribution
        lamb[i] = ((n-i)*t_pv) / (math.sqrt((n-i-1+t_pv**2)*(n-i+1)))
        
        #### Find the exact number of outliers: largest 'i' such that R_i > lambda_i ####
        #print(i, R[i], lamb[i])
        if ( (not pd.isna(R[i])) & (not pd.isna(lamb[i])) ): # stats.t.ppf can produces nans
            if (R[i] > lamb[i]):
                m = i + 1
    vals = pd.DataFrame.from_dict(dict(zip(['NumOutliers','TestStatistic','CriticalValue'], 
                                           [range(r),R,lamb])))
    
    #print (m)
    outlier_rank = np.zeros(shape = n) - 1
    if ( m> -1):
        for i in range(m):
            outlier_rank[np.where(obs==outlier_val[i])] = i
            #print(outlier_rank[np.where(obs==outlier_val[i])])
    
    res = dict()
    res['table'] = vals
    res['num_outliers'] = sum(outlier_rank!= -1)
    res['outlier_rank'] = outlier_rank
    return res

In [ ]:
#assuming no more than 5% of my data will be an outliers
r = round(len(noise) * 0.05)

#the critical value of my test
alpha = 0.1
gesd_result = gesd(noise, alpha,"No", r)

The object returned by `gesd` will tell you how many outliers it detected in its `num_outliers` property

In [ ]:
gesd_result['num_outliers']

Additionally, in its `table` property, the `gesd` object will produce a table for each row $i$, which compares $R_i$ (the test statistics) to $\lambda_i$ (the critical value)

In [ ]:
gesd_result['table']

Finally, in its `outlier_rank` property, the `gesd` object will produce a vector (in order of the original data points) which contains the rank of the outlier (or -1 if the point is not an outlier). 

In [ ]:
gesd_result['outlier_rank']

In [ ]:
detected = gesd_result['outlier_rank']>-1

In [ ]:
detected

### Plotting the detected object

We can use the following helper function to plot the original time series and highlight the outliers detected. 

In [ ]:
import datetime as dt
import matplotlib.dates as mdates
def plot_detected(df, detected, column_name):
    
    x = df.index
    y = df[column_name].values
    fig, ax = plt.subplots(figsize = (10,6))
    ax.plot_date(x, y, linestyle = 'solid')
    
    detected_ind = np.where(detected)[0]
    
    for i in range(len(detected_ind)):
        print(detected_ind[i])
        ax.annotate('Outlier', (mdates.date2num(x[detected_ind[i]]), y[detected_ind[i]]), xytext=(60, 60), 
            textcoords='offset points', arrowprops=dict(arrowstyle='-|>'))

In [ ]:
#Highlighting the original points whose noise is extreme
plot_detected(elec_df, detected, column_name = 'Value')